In [2]:
# !pip install autogluon
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [3]:
# 1️⃣ Cargar los datasets
sellin = pd.read_csv("datasets/sell-in.csv", sep='\t')
productos = pd.read_csv("datasets/tb_productos.csv", sep='\t')
stocks = pd.read_csv("datasets/tb_stocks.csv", sep='\t')
productos_a_predecir = pd.read_csv("datasets/product_id_apredecir201912.csv")

# Crear DataFrame principal
df = sellin.merge(productos, on="product_id", how="left").merge(stocks, on=["product_id", "periodo"], how="left")
df['periodo_dt'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

# Sumarizar ventas por periodo y producto
ventas = df.groupby(['periodo', 'product_id']).agg({'tn':'sum'}).reset_index()
ventas['periodo_dt'] = pd.to_datetime(ventas['periodo'].astype(str), format='%Y%m')

# Determinar min y max periodo por producto
vida_producto = ventas.groupby('product_id')['periodo_dt'].agg(['min','max']).reset_index()

# Crear DataFrame expandido con todos <periodo, producto>
all_periods = []
for _, row in vida_producto.iterrows():
    pid = row['product_id']
    periods = pd.date_range(row['min'], row['max'], freq='MS')
    for p in periods:
        all_periods.append((pid, p))
df_full = pd.DataFrame(all_periods, columns=['product_id','periodo_dt'])

# Unir con ventas reales
ventas = ventas[['product_id','periodo_dt','tn']]
df_pp = df_full.merge(ventas, on=['product_id','periodo_dt'], how='left')
df_pp['tn'] = df_pp['tn'].fillna(0)
df_pp['ds'] = df_pp['periodo_dt']
df_pp['y'] = df_pp['tn']



In [6]:
df_pp

,product_id,periodo_dt,tn,ds,y
0,20001,2017-01-01,934.77222,2017-01-01,934.77222
1,20001,2017-02-01,798.01620,2017-02-01,798.01620
2,20001,2017-03-01,1303.35771,2017-03-01,1303.35771
3,20001,2017-04-01,1069.96130,2017-04-01,1069.96130
4,20001,2017-05-01,1502.20132,2017-05-01,1502.20132
...,...,...,...,...,...
31517,21295,2017-01-01,0.00699,2017-01-01,0.00699
31518,21296,2017-08-01,0.00651,2017-08-01,0.00651
31519,21297,2017-01-01,0.00579,2017-01-01,0.00579
31520,21298,2017-08-01,0.00573,2017-08-01,0.00573


In [8]:
df_pp.rename(columns={'product_id': 'item_id', 'ds': 'timestamp', 'y': 'target'}, inplace=True)



In [10]:
df_pp.drop(columns=['periodo_dt', 'tn'], inplace=True)

In [11]:
data = TimeSeriesDataFrame(df_pp)
data

target
item_id timestamp             
20001   2017-01-01   934.77222
        2017-02-01   798.01620
        2017-03-01  1303.35771
        2017-04-01  1069.96130
        2017-05-01  1502.20132
...                        ...
21295   2017-01-01     0.00699
21296   2017-08-01     0.00651
21297   2017-01-01     0.00579
21298   2017-08-01     0.00573
21299   2017-08-01     0.00546

[31522 rows x 1 columns]

In [15]:
# Filtrar solo productos a predecir
productos_ids = productos_a_predecir['product_id'].unique()

In [12]:
predictor = TimeSeriesPredictor(target='target', prediction_length=2, freq="M").fit(data)
predictions = predictor.predict(data)

Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Usuario\Documents\Universidad\austral\2025\Lab3\Lab3-MCD\AutogluonModels\ag-20250530_214317'
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.4
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          4
GPU Count:          0
Memory Avail:       3.73 GB / 15.89 GB (23.4%)
Disk Space Avail:   422.86 GB / 893.49 GB (47.3%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

train_data with frequency 'None' has been resampled to frequency 'ME'.

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/821M [00:00<?, ?B/s]

	-0.2871       = Validation score (-WQL)
	13.35   s     = Training runtime
	30.68   s     = Validation (prediction) runtime
Training timeseries model ChronosFineTuned[bolt_small]. 
	Skipping covariate_regressor since the dataset contains no covariates or static features.


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/191M [00:00<?, ?B/s]

	Fine-tuning on the CPU detected. We recommend using a GPU for faster fine-tuning of Chronos.
	TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'
Training timeseries model TemporalFusionTransformer. 
	-0.2665       = Validation score (-WQL)
	374.04  s     = Training runtime
	1.09    s     = Validation (prediction) runtime
Training timeseries model DeepAR. 
	-0.2546       = Validation score (-WQL)
	138.71  s     = Training runtime
	2.10    s     = Validation (prediction) runtime
Training timeseries model PatchTST. 
	-0.2695       = Validation score (-WQL)
	88.35   s     = Training runtime
	0.67    s     = Validation (prediction) runtime
Training timeseries model TiDE. 
	-0.2816       = Validation score (-WQL)
	366.73  s     = Training runtime
	1.70    s     = Validation (prediction) runtime
Fitting simple weighted ensemble.
	Ensemble weights: {'AutoETS': 0.06, 'DeepAR': 0.71, 'DynamicOptimizedTheta': 0.03, 'RecursiveTabular': 0.02, 'TemporalFusionTrans

In [13]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2020-02-29"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
predictions_v1.head(5)

,product_id,tn
1,20001,1302.976815
3,20002,1102.488131
5,20003,666.269509
7,20004,550.978395
9,20005,577.405090


In [17]:
predictions_v1 = predictions_v1[predictions_v1['product_id'].isin(productos_ids)]

In [21]:
predictions_v1.to_csv("./kaggle/predictions_autogluon.csv", index=False)